In [ ]:
# environment: thesisPlayground_pointClouds_env

I was helped by these discussions / resources:
- https://github.com/apple/ml-hypersim/blob/main/contrib/mikeroberts3000/jupyter/00_projecting_points_into_hypersim_images.ipynb
- https://github.com/apple/ml-hypersim/issues/44

In [1]:
import h5py
import numpy as np
import open3d as o3d
import pandas as pd
import glob
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Using positions - Point cloud from one image using the camera-space (use M_cam_from_world)

Remark that everything was prepared as if the code had to handle more than one image. However, working in the camera-space would require some modifications in the code, so at the end this section just works to render an image, I guess. A revision is needed in any case; I kept the code because maybe it will turn out useful.

In [ ]:
def load_hdf5_file(file_paths, dataset_key):
    data = []
    for file in file_paths:
        with h5py.File(file, 'r') as f:
            data.append(np.array(f[dataset_key]))
    return data


def build_M_cam_from_world(frame_id, camera_dir):

    camera_positions_hdf5_file    = os.path.join(camera_dir, "camera_keyframe_positions.hdf5")
    camera_orientations_hdf5_file = os.path.join(camera_dir, "camera_keyframe_orientations.hdf5")

    with h5py.File(camera_positions_hdf5_file,    "r") as f: camera_positions    = f["dataset"][:]
    with h5py.File(camera_orientations_hdf5_file, "r") as f: camera_orientations = f["dataset"][:]

    # get position and rotation matrix for Hypersim image
    camera_position_world = camera_positions[frame_id]
    R_world_from_cam      = camera_orientations[frame_id]

    t_world_from_cam = np.array(camera_position_world).reshape((3, 1))
    R_cam_from_world = np.array(R_world_from_cam).T
    t_cam_from_world = -R_cam_from_world @ t_world_from_cam

    M_cam_from_world = np.block([[R_cam_from_world, t_cam_from_world],
                                 [np.zeros((1, 3)), np.array([[1.0]])]])
    
    return M_cam_from_world
    

def extract_frame_id(filename):
    base = os.path.basename(filename)
    frame_id = int(base.split('.')[1])
    return frame_id


def generate_point_cloud(image_files, positions, camera_dir):
    point_clouds = []

    for img_file, position in zip(image_files, positions):
        # Extract frame ID from the image filename
        frame_id = extract_frame_id(img_file)

        # Build the M_cam_from_world matrix for the current frame
        M_cam_from_world = build_M_cam_from_world(frame_id, camera_dir)

        # Extracting 3D coordinates of each pixel from positions
        X = position[:, :, 0]
        Y = position[:, :, 1]
        Z = position[:, :, 2]

        # Stack X, Y, Z to get point cloud
        point_cloud = np.stack((X, Y, Z), axis=-1)

        # Reshape point cloud to flatten the array
        point_cloud_flat = point_cloud.reshape(-1, 3)

        # Convert world-space points to camera-space points
        homogeneous_points = np.hstack((point_cloud_flat, np.ones((point_cloud_flat.shape[0], 1))))
        camera_space_points = np.dot(M_cam_from_world, homogeneous_points.T).T[:, :3]

        # Convert camera-space points to Open3D point cloud format
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(camera_space_points)

        # Append point cloud to list
        point_clouds.append(pcd)

    return point_clouds


def render_point_cloud(point_cloud):

    pcd = point_cloud

    # Create a visualization window
    vis = o3d.visualization.Visualizer()

    # Add the point cloud to the visualization window
    vis.create_window()
    vis.add_geometry(pcd)

    # Set the render options (optional)
    render_options = vis.get_render_option()
    render_options.point_size = 2  # Adjust the size of the points

    # Render the visualization
    vis.run()

    # Close the visualization window
    vis.destroy_window()


In [ ]:
base_path = '/local/home/gmarsich/data2TB/Hypersim/evermotion_dataset/scenes'
scene = 'ai_007_008'  # name of the scene, with format ai_VVV_NNN
cam_xx = 'cam_00'

# Get directory for info
camera_dir = os.path.join(base_path, scene, "_detail", cam_xx)

# Get list of image and position HDF5 files
image_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_final_hdf5', '*.color.hdf5')))
position_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_geometry_hdf5', '*.position.hdf5')))

# HERE WE SELECT JUST ONE IMAGE
frame = 0
image_files = [image_files[frame]]
position_files = [position_files[frame]]


# Ensure the number of image and position files match
if len(image_files) != len(position_files):
    raise ValueError("The number of image files and position files do not match.")

In [ ]:
# Load images and position data
images = load_hdf5_file(image_files, 'dataset')
positions = load_hdf5_file(position_files, 'dataset')

# Generate point clouds for each view
point_clouds = generate_point_cloud(image_files, positions, camera_dir)

cloud = point_clouds[0]
print(cloud)

render_point_cloud(point_clouds[0])

# Using positions - Point cloud from more images using the world-space
### Colour given by the height from the floor

In [2]:
def load_hdf5_file(file_paths, dataset_key):
    data = []
    for file in file_paths:
        with h5py.File(file, 'r') as f:
            data.append(np.array(f[dataset_key]))
    return data


def generate_point_cloud(images, positions):
    point_clouds = []

    for position in positions:
        # Extracting 3D coordinates of each pixel from positions
        X = position[:, :, 0]
        Y = position[:, :, 1]
        Z = position[:, :, 2]

        # Stack X, Y, Z to get point cloud
        point_cloud = np.stack((X, Y, Z), axis=-1)

        # Convert numpy array to Open3D point cloud format
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud.reshape(-1, 3))

        # Append point cloud to list
        point_clouds.append(pcd)

    return point_clouds


def combine_point_clouds(point_clouds):
    # Combine all point clouds into a single point cloud
    combined_cloud = o3d.geometry.PointCloud()
    for pcd in point_clouds:
        combined_cloud += pcd

    return combined_cloud


def render_point_cloud(point_cloud):

    pcd = point_cloud

    # Create a visualization window
    vis = o3d.visualization.Visualizer()

    # Add the point cloud to the visualization window
    vis.create_window()
    vis.add_geometry(pcd)

    # Set the render options (optional)
    render_options = vis.get_render_option()
    render_options.point_size = 2  # Adjust the size of the points

    # Render the visualization
    vis.run()

    # Close the visualization window
    vis.destroy_window()


In [3]:
base_path = '/local/home/gmarsich/data2TB/Hypersim/evermotion_dataset/scenes'
scene = 'ai_007_008'  # name of the scene, with format ai_VVV_NNN
cam_xx = 'cam_00'

# Get list of image and position HDF5 files
image_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_final_hdf5', '*.color.hdf5')))
position_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_geometry_hdf5', '*.position.hdf5')))

# Ensure the number of image and position files match
if len(image_files) != len(position_files):
    raise ValueError("The number of image files and position files do not match.")

In [4]:
# Load images and position data
images = load_hdf5_file(image_files, 'dataset')
positions = load_hdf5_file(position_files, 'dataset')

# Generate point clouds for each view
point_clouds = generate_point_cloud(images, positions)

# Combine all individual point clouds into a single point cloud
combined_point_cloud = combine_point_clouds(point_clouds)

o3d.io.write_point_cloud("point_cloud.ply", combined_point_cloud) # to save the point cloud as .ply file

# Render the visualization of the combined point cloud
render_point_cloud(combined_point_cloud)

In [ ]:
# # How to open a point cloud

# pcd = o3d.io.read_point_cloud("/local/home/gmarsich/Desktop/Thesis/0Code_playground/pointClouds/pointClouds_ChatGPT/point_cloud.ply")
# render_point_cloud(pcd)

# Using positions - Point cloud from more images using the world-space
### Colour given by the objects (i.e., by the *color.hdf5 files)
Remark that no tonemapping has been applied.

In [ ]:

def load_hdf5_file(file_paths, dataset_key):
    data = []
    for file in file_paths:
        with h5py.File(file, 'r') as f:
            data.append(np.array(f[dataset_key]))
    return data


def generate_point_cloud(images, positions):
    point_clouds = []

    for i, (image, position) in enumerate(zip(images, positions)):
        # Extracting 3D coordinates of each pixel from positions
        X = position[:, :, 0]
        Y = position[:, :, 1]
        Z = position[:, :, 2]

        # Extracting color information from images
        R = image[:, :, 0]
        G = image[:, :, 1]
        B = image[:, :, 2]


        # # Uncomment the following to have different colors
        
        # # Debug statements to check the range of the raw image data
        # print(f"Raw image {i} ranges: R min {R.min()}, max {R.max()}, G min {G.min()}, max {G.max()}, B min {B.min()}, max {B.max()}")

        # # Find the maximum value across all color channels to normalize the colors
        # max_value = max(R.max(), G.max(), B.max())
        # if max_value > 0:
        #     R = R / max_value
        #     G = G / max_value
        #     B = B / max_value

        # Stack X, Y, Z to get point cloud coordinates
        point_cloud = np.stack((X, Y, Z), axis=-1)
        # Stack R, G, B to get point cloud colors
        colors = np.stack((R, G, B), axis=-1)

        # Convert numpy array to Open3D point cloud format
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud.reshape(-1, 3))
        pcd.colors = o3d.utility.Vector3dVector(colors.reshape(-1, 3))

        # Append point cloud to list
        point_clouds.append(pcd)

    return point_clouds


def combine_point_clouds(point_clouds):
    # Combine all point clouds into a single point cloud
    combined_cloud = o3d.geometry.PointCloud()
    for pcd in point_clouds:
        combined_cloud += pcd

    return combined_cloud


def render_point_cloud(point_cloud):
    pcd = point_cloud

    # Create a visualization window
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    # Set the render options (optional)
    render_options = vis.get_render_option()
    render_options.point_size = 2  # Adjust the size of the points

    # Render the visualization
    vis.run()

    # Close the visualization window
    vis.destroy_window()


In [ ]:
base_path = '/local/home/gmarsich/data2TB/Hypersim/evermotion_dataset/scenes'
scene = 'ai_007_008'
cam_xx = 'cam_00'

# Get list of image and position HDF5 files
image_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_final_hdf5', '*.color.hdf5')))
position_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_geometry_hdf5', '*.position.hdf5')))

# Ensure the number of image and position files match
if len(image_files) != len(position_files):
    raise ValueError("The number of image files and position files do not match.")

In [ ]:
# Load images and position data
images = load_hdf5_file(image_files, 'dataset')
positions = load_hdf5_file(position_files, 'dataset')

# Generate point clouds for each view
point_clouds = generate_point_cloud(images, positions)

# Combine all individual point clouds into a single point cloud
combined_point_cloud = combine_point_clouds(point_clouds)

# Render the visualization of the combined point cloud
render_point_cloud(combined_point_cloud)


# Using positions - Point cloud from more images using the world-space
### Colour given by a tonemapping

Here one first needs to preprocess the `*color.hdf5 files`, applying a tonemap. I considered the same tonemap as the one used by Hypersim (and described in this code: https://github.com/apple/ml-hypersim/blob/main/code/python/tools/scene_generate_images_tonemap.py). I created a file `apply_tonemap.py` that converts `.color.hdf5` files into hdf5 files with the tonemap, and saves them.

In [ ]:
# import apply_tonemap # uncomment if you need to get the hdf5 files with tonemap

base_path = '/local/home/gmarsich/data2TB/Hypersim/evermotion_dataset/scenes'
scene = 'ai_007_008'
cam_xx = 'cam_00'

in_dir = os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_final_hdf5')
out_dir = os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_final_hdf5', 'scene_' + cam_xx + '_final_hdf5_TONEMAP')


# apply_tonemap.apply_tonemapping_to_directory(in_dir, out_dir) # uncomment if you need to get the hdf5 files with tonemap

# Get list of image and position HDF5 files
image_files = sorted(glob.glob(os.path.join(out_dir, '*.color.hdf5')))
position_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_geometry_hdf5', '*.position.hdf5')))

# Ensure the number of image and position files match
if len(image_files) != len(position_files):
    raise ValueError("The number of image files and position files do not match.")


In [ ]:

def load_hdf5_file(file_paths, dataset_key):
    data = []
    for file in file_paths:
        with h5py.File(file, 'r') as f:
            data.append(np.array(f[dataset_key]))
    return data


def generate_point_cloud(images, positions):
    point_clouds = []

    for i, (image, position) in enumerate(zip(images, positions)):
        # Extracting 3D coordinates of each pixel from positions
        X = position[:, :, 0]
        Y = position[:, :, 1]
        Z = position[:, :, 2]

        # Extracting color information from images
        R = image[:, :, 0]
        G = image[:, :, 1]
        B = image[:, :, 2]

        # Debug statements to check the range of the raw image data
        print(f"Raw image {i} ranges: R min {R.min()}, max {R.max()}, G min {G.min()}, max {G.max()}, B min {B.min()}, max {B.max()}")

        # Find the maximum value across all color channels to normalize the colors
        max_value = max(R.max(), G.max(), B.max())
        if max_value > 0:
            R = R / max_value
            G = G / max_value
            B = B / max_value

        # Stack X, Y, Z to get point cloud coordinates
        point_cloud = np.stack((X, Y, Z), axis=-1)
        # Stack R, G, B to get point cloud colors
        colors = np.stack((R, G, B), axis=-1)

        # Convert numpy array to Open3D point cloud format
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud.reshape(-1, 3))
        pcd.colors = o3d.utility.Vector3dVector(colors.reshape(-1, 3))

        # Append point cloud to list
        point_clouds.append(pcd)

    return point_clouds


def combine_point_clouds(point_clouds):
    # Combine all point clouds into a single point cloud
    combined_cloud = o3d.geometry.PointCloud()
    for pcd in point_clouds:
        combined_cloud += pcd

    return combined_cloud


def render_point_cloud(point_cloud):
    pcd = point_cloud

    # Create a visualization window
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    # Set the render options (optional)
    render_options = vis.get_render_option()
    render_options.point_size = 2  # Adjust the size of the points

    # Render the visualization
    vis.run()

    # Close the visualization window
    vis.destroy_window()


In [ ]:
# Load images and position data
images = load_hdf5_file(image_files, 'tonemapped_rgb')
positions = load_hdf5_file(position_files, 'dataset')

# Generate point clouds for each view
point_clouds = generate_point_cloud(images, positions)

# Combine all individual point clouds into a single point cloud
combined_point_cloud = combine_point_clouds(point_clouds)

# Render the visualization of the combined point cloud
render_point_cloud(combined_point_cloud)
